In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [11]:
import zipfile
import os
from datasets import Dataset,DatasetDict
from transformers import AutoTokenizer, TrainingArguments , Trainer, DataCollatorForSeq2Seq , AutoModelForSeq2SeqLM


In [3]:
def extraction(path,dest):
  with zipfile.ZipFile(path, 'r') as zip_ref:
      zip_ref.extractall(dest)

  print(os.listdir(dest))

In [4]:
extraction('Full_MILDSum_Dataset-20241105T105654Z-001.zip','dataset')

['Full_MILDSum_Dataset']


In [5]:
extraction('dataset/dataset/MILDSum_train_2185.zip','train')
extraction('dataset/dataset/MILDSum_test_468.zip','test')
extraction('dataset/dataset/MILDSum_val_469.zip','val')

['MILDSum_train_2185']
['MILDSum_test_468']
['MILDSum_val_469']


In [7]:
def dataset_sorter(path):
    judgement = []
    summary = []
    ids = []

    for root, _ , files in os.walk(path):

        if files:
            subdirectory_name = os.path.basename(root)
            #print(subdirectory_name)
            file_path1 = os.path.join(root,files[2])
            file_path2 = os.path.join(root,files[1])
            # print(file_path1)
            # print(file_path2)

            with open(file_path1,"r") as fp:
                content = fp.read()
                judgement.append(content)

            with open(file_path2,"r") as fp:
                content = fp.read()
                summary.append(content)

            ids.append(int(subdirectory_name))


    assert len(judgement) == len(summary), "Mismatch in articles and highlights"

    # Create a dictionary to store the data
    data_dict = {
        'judgement': judgement,
        'summary': summary,
        'id': ids
    }

    # Load data into a Hugging Face Dataset


    # Display dataset info
    return data_dict

In [8]:
d_train = dataset_sorter("train/MILDSum_train_2185")
d_test = dataset_sorter("test/MILDSum_test_468")
d_val = dataset_sorter("val/MILDSum_val_469")

In [9]:
len(d_train['judgement'])

2185

In [10]:
dataset = {}
dataset['train'] = d_train
dataset['test'] = d_test
dataset['validation'] = d_val

In [12]:
model_name = 'google-t5/t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [13]:
tokenizer.model_max_length = 1024
model.config.max_length = 1024

In [14]:
def filter_and_reindex(data, model_name,dir,tokenizer, token_limit=1024):
    """
    Filters judgements and summaries under the token limit,
    appends them to new lists, and creates new IDs.

    Args:
        data (dict): The dataset containing judgements and summaries.
        model_name (str): The name of the pre-trained tokenizer.
        token_limit (int): The token limit (default: 4096).

    Returns:
        dict: A dictionary containing the filtered judgements, summaries, and new IDs.
    """
    filtered_judgements = []
    filtered_summaries = []
    new_ids = []
    current_id = 0

    for judgement, summary in zip(data[dir]['judgement'], data[dir]['summary']):
        judgement_tokens = len(tokenizer.encode(judgement, truncation=True, max_length=token_limit))
        summary_tokens = len(tokenizer.encode(summary, truncation=True, max_length=token_limit))

        if judgement_tokens < token_limit and summary_tokens < token_limit:
            filtered_judgements.append(judgement)
            filtered_summaries.append(summary)
            new_ids.append(current_id)
            current_id += 1

    return {
        'judgement': filtered_judgements,
        'summary': filtered_summaries,
        'id': new_ids
    }



In [16]:
filtered_data_train = filter_and_reindex(dataset, model_name,'train',tokenizer)

print(f"Number of filtered judgements: {len(filtered_data_train['judgement'])}")
print(f"Number of filtered summaries: {len(filtered_data_train['summary'])}")
print(f"New IDs: {filtered_data_train['id'][:10]}")  # Print first 10 new IDs

filtered_data_test = filter_and_reindex(dataset, model_name,'test',tokenizer)

print(f"Number of filtered judgements: {len(filtered_data_test['judgement'])}")
print(f"Number of filtered summaries: {len(filtered_data_test['summary'])}")
print(f"New IDs: {filtered_data_test['id'][:10]}")  # Print first 10 new IDs

filtered_data_val = filter_and_reindex(dataset, model_name,'validation',tokenizer)

print(f"Number of filtered judgements: {len(filtered_data_val['judgement'])}")
print(f"Number of filtered summaries: {len(filtered_data_val['summary'])}")
print(f"New IDs: {filtered_data_val['id'][:10]}")  # Print first 10 new IDs

Number of filtered judgements: 220
Number of filtered summaries: 220
New IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Number of filtered judgements: 46
Number of filtered summaries: 46
New IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Number of filtered judgements: 48
Number of filtered summaries: 48
New IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [17]:
# Convert each split into a Dataset
train_dataset = Dataset.from_dict(filtered_data_train)
test_dataset = Dataset.from_dict(filtered_data_test)
validation_dataset = Dataset.from_dict(filtered_data_val)

# Combine into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

In [18]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["judgement"],
        max_length=1024,
        padding="max_length",
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=1024,
            padding="max_length",
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/220 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [19]:
training_args = TrainingArguments(
    output_dir="./T5-1024",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    push_to_hub=False,
    gradient_accumulation_steps=2,
    fp16=True,
    save_total_limit=1,  # Keep only the best checkpoint
    load_best_model_at_end=True,
    save_steps=1000,

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,

)

<ipython-input-20-1cd026ab8582>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,1.469747
2,No log,1.320234


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 1024}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=110, training_loss=5.967700750177557, metrics={'train_runtime': 371.1191, 'train_samples_per_second': 1.186, 'train_steps_per_second': 0.296, 'total_flos': 535882943692800.0, 'train_loss': 5.967700750177557, 'epoch': 2.0})

In [22]:
trainer.save_model('t5base_1024_v1')

In [24]:
!tar -czvf model.tar.gz t5base_1024_v1

t5base_1024_v1/
t5base_1024_v1/spiece.model
t5base_1024_v1/training_args.bin
t5base_1024_v1/tokenizer.json
t5base_1024_v1/generation_config.json
t5base_1024_v1/config.json
t5base_1024_v1/tokenizer_config.json
t5base_1024_v1/special_tokens_map.json
t5base_1024_v1/model.safetensors
